In [ ]:
!pip install git+https://github.com/taehoonlee/tensornets.git

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensornets as nets
import cv2
import numpy as np

import time

import matplotlib.pyplot as plt

inputs = tf.placeholder(tf.float32, [None, 416, 416, 3])
model = nets.YOLOv3COCO(inputs, nets.Darknet19)
# model = nets.YOLOv2(inputs, nets.Darknet19)
#frame=cv2.imread("D://pyworks//yolo//truck.jpg",1)
from google.colab import drive
drive.mount('/content/drive')

  Cloning https://github.com/taehoonlee/tensornets.git to /tmp/pip-req-build-iaod_ae2
  Running command git clone -q https://github.com/taehoonlee/tensornets.git /tmp/pip-req-build-iaod_ae2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for tensornets: filename=tensornets-0.4.6-cp37-cp37m-linux_x86_64.whl size=894918 sha256=1d9366a1e01fbbf0639915775ecb385bfa64ddd76183c5779c1866e0035edd4f
  Stored in directory: /tmp/pip-ephem-wheel-cache-pjkpnv9i/wheels/4e/46/81/c404dda2ae2b6d8aba5a436c5c749c63dc7c95aa08481d2bc9
Successfully built tensornets
Instructions for updating:
non-resource variables are not supported in the long term


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
classes={'0':'person'}
list_of_classes=[0]
frames_count=0
with tf.Session() as sess:
    sess.run(model.pretrained())
#"D://pyworks//yolo//videoplayback.mp4"    

    # uploaded = files.upload()
    cap = cv2.VideoCapture('/content/drive/My Drive/StoreFeed.mp4')

    ret,frame1=cap.read() #ret is boolean
    ret,frame2=cap.read()

    fourcc = cv2.VideoWriter_fourcc(*'XVID') #fourcc code defines the format of o/p video
    Output = cv2.VideoWriter('/content/drive/My Drive/OptimizedCamDe.avi', fourcc, 20.0, (416,416), True)  #20.0 = fps, 416x416 = frame size
    
    print('Execution Starts..')
    
    count=0
    init_time=time.time()
    start_time=time.time()
    try:

      while(cap.isOpened()):
          if frames_count%2==0:
            frames_count+=1
            
            frame1 = frame2
            ret, frame2 = cap.read()
            
            continue
          diff=cv2.absdiff(frame1,frame2)  #calculates diff of two consecutive frame
          gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) #converts diff in grayscale
          blur = cv2.GaussianBlur(gray,(9,9),0) # smoothens the edges and makes countours more easily detectable 
          _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY) #
          dilated = cv2.dilate(thresh, None, iterations = 3) #expands the nearest 3 pixels, increases the white area to define countours properly
          contours, hierarchy = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #saves co-ordinates where countour is identified
          img=cv2.resize(frame1,(416,416)) #resized for YOLO model

          if contours is not None:
            flagMotion=False

            for contour in contours:
              if cv2.contourArea(contour)>800:
                flagMotion=True
                break
          
          if flagMotion:
            start_time=time.time()
            # ret, frame = cap.read()
            # print(frame.shape)
            # img=cv2.resize(frame,(416,416))
            imge=np.array(img).reshape(-1,416,416,3) 
            
            preds = sess.run(model.preds, {inputs: model.preprocess(imge)}) #list of co-ordinates where person is detected and also stores the class of object
        
            # print("--- %s seconds ---" % (time.time() - init_time)) 
            boxes = model.get_boxes(preds, imge.shape[1:3]) #extracting co-ordinates
            # cv2.namedWindow('image',cv2.WINDOW_NORMAL)

            # cv2.resizeWindow('image', 700,700)
            #print("--- %s seconds ---" % (time.time() - start_time)) 
            boxes1=np.array(boxes)
            for j in list_of_classes:
                count =0
                if str(j) in classes:
                    lab=classes[str(j)] #label
                if len(boxes1) !=0:
                    for i in range(len(boxes1[j])):
                        box=boxes1[j][i] 
                        if boxes1[j][i][4]>=.40:  ##         
                            count += 1
                            cv2.rectangle(img,(box[0],box[1]),(box[2],box[3]),(0,255,0),1)  #Color: B,G,R; thickness=1
    #                        cv2.putText(img, lab, (box[0],box[1]), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), lineType=cv2.LINE_AA)
                # print(lab,": ",count)
                print("Person Detected at : ", (time.time()-start_time)," s")
                
            else:
              pass
                  
            # cv2.imshow("image",img)  
          Output.write(img)
          print('Frame: ',frames_count)
          frames_count+=1

          # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
          # plt.show()

          frame1 = frame2
          ret, frame2 = cap.read()

          if cv2.waitKey(1) & 0xFF == ord('q'): #interrupt
              break          

    except:
      print('Execution Successful\n')
      print('Time take for Execution: ',(time.time() - init_time),'seconds')

cap.release()
cv2.destroyAllWindows()    



Execution Starts..
Frame:  1
Frame:  3
Frame:  5
Frame:  7
Frame:  9
Frame:  11
Frame:  13
Frame:  15
Frame:  17
Frame:  19
Frame:  21
Frame:  23
Frame:  25
Frame:  27
Frame:  29
Frame:  31
Frame:  33
Frame:  35
Frame:  37
Frame:  39
Frame:  41
Frame:  43
Frame:  45
Frame:  47
Frame:  49
Frame:  51
Frame:  53
Frame:  55
Frame:  57
Frame:  59
Frame:  61
Frame:  63
Frame:  65
Frame:  67
Frame:  69
Frame:  71
Frame:  73
Frame:  75
Frame:  77
Frame:  79
Frame:  81
Frame:  83
Frame:  85
Frame:  87
Frame:  89
Frame:  91
Frame:  93
Frame:  95
Frame:  97
Frame:  99
Frame:  101
Frame:  103
Frame:  105
Frame:  107
Frame:  109
Frame:  111
Frame:  113
Frame:  115
Frame:  117
Frame:  119
Frame:  121
Frame:  123
Frame:  125
Frame:  127
Frame:  129
Frame:  131
Frame:  133
Frame:  135
Frame:  137


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Person Detected at :  1.3160295486450195  s
Frame:  139
Frame:  141
Frame:  143
Frame:  145
Person Detected at :  0.9844789505004883  s
Frame:  147
Person Detected at :  0.9604690074920654  s
Frame:  149
Person Detected at :  0.9640636444091797  s
Frame:  151
Person Detected at :  0.9630799293518066  s
Frame:  153
Person Detected at :  0.9455223083496094  s
Frame:  155
Person Detected at :  0.955089807510376  s
Frame:  157
Person Detected at :  0.9825689792633057  s
Frame:  159
Person Detected at :  0.9313638210296631  s
Frame:  161
Person Detected at :  0.9517214298248291  s
Frame:  163
Person Detected at :  0.954362154006958  s
Frame:  165
Person Detected at :  0.9452922344207764  s
Frame:  167
Person Detected at :  0.9421942234039307  s
Frame:  169
Person Detected at :  0.9562647342681885  s
Frame:  171
Person Detected at :  0.9305741786956787  s
Frame:  173
Person Detected at :  0.935218334197998  s
Frame:  175
Person Detected at :  0.928638219833374  s
Frame:  177
Person Detected 